In [3]:
from keras.models  import load_model
import keras.backend as K
from sklearn.preprocessing import normalize
import pickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer
from colors import color
import pandas as pd
import numpy as np

aspects = [str(i) for i in range(4)] #app, aroma, palate,taste

In [5]:
with open('../../beer_data/beer_vec_ds_df10.p', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    vec = u.load()

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

ds = pd.read_csv('../../beer_data/beer_ds.csv')

from ast import literal_eval as make_tuple
ds['bits'] = ds['bits'].map(lambda s : make_tuple(s))

aspect_columns = sorted(['review/appearance', 'review/taste', 'review/aroma', 'review/palate'])
aspects = [str(i) for i in range(4)]

train_idxs, val_idxs = train_test_split(ds.index, stratify=ds['bits'], train_size=0.9, random_state=1337)

/home/sarthak/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [10]:
idxs = val_idxs
H = {}
for i in range(4) :
    H[str(i)] = np.zeros((len(idxs), len(idxs)))
    a0 = set(ds[ds[aspect_columns[i]] == 0].index) & set(idxs)
    a1 = set(ds[ds[aspect_columns[i]] == 1].index) & set(idxs)
    a0 = list(map(lambda s : list(idxs).index(s), a0))
    a1 = list(map(lambda s : list(idxs).index(s), a1))
    for j in a0 :
        H[str(i)][j, a0] = 1
    for j in a1 :
        H[str(i)][j, a1] = 1

    H[str(i)][np.arange(len(idxs)), np.arange(len(idxs))] = 0

In [11]:
model = load_model('../store/weights/beer/Gated2CNNModelSatJan615:09:332018/loss.h5', 
                   custom_objects={'contrastive_loss' : lambda a,b:b, '<lambda>' : lambda a,b : b})

In [12]:
model.get_layer('pred_0').get_layer('pool_0').summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 316)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 316, 300)     4407000     input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 316, 1)       0           input_1[0][0]                    
__________________________________________________________________________________________________
multiply_1 (Multiply)           (None, 316, 300)     0           embedding[0][0]                  
                                                                 lambda_1[0][0]                   
__________

In [13]:
model_c1 = {}
for aspect in aspects :
    model_c1[aspect] = K.function(model.get_layer('pred_'+aspect).get_layer('pool_'+aspect).inputs, 
                          [model.get_layer('pred_'+aspect).get_layer('pool_'+aspect).layers[16].output])

In [14]:
X = vec.X[val_idxs]

In [16]:
filters_pico = {k:[] for k in aspects}
#X = X[-2:]
i, bs = 0, 100
while i*bs < len(X):
    for aspect in aspects :
        val = model_c1[aspect]([X[i*bs:(i+1)*bs]])[0]#[:, :, :]
        val = val.mean(axis=-1)
        val = np.dstack((X[i*bs:(i+1)*bs], val))
        filters_pico[aspect].append(val)
    i+=1
        
for aspect in aspects :
    filters_pico[aspect] = np.concatenate(filters_pico[aspect])
#     for s in range(len(filters_pico[aspect])) :
#         nz = sum([1 for i in filters_pico[aspect][s, :, 0] if i == 0])
#         filters_pico[aspect][s, :nz, 1] = 1.
#         filters_pico[aspect][s, :, 1] = np.convolve(filters_pico[aspect][s, :, 1], 
#                                                     [1./5 for _ in range(5)], mode='same')
    #print filters_pico[aspect].shape

In [18]:
from random import random, randint
for v in range(2) :
    s = randint(0, len(filters_pico['3'])-1)

    nz = sum([1 for i in filters_pico['3'][s, :, 0] if i == 0])
    gate0 = filters_pico['0'][s, :, 1]
    gate1 = filters_pico['1'][s, :, 1]
    gate2 = filters_pico['2'][s, :, 1]
    gate3 = filters_pico['3'][s, :, 1]

    gate0[:nz] = 1.
#     gate1[:nz] = 1.
#     gate2[:nz] = 1.
#     gate3[:nz] = 1.
    gate0 = np.convolve(gate0, [1./5 for _ in range(5)], mode='same')
    gate1 = np.convolve(gate1, [1./5 for _ in range(5)], mode='same')
    gate2 = np.convolve(gate2, [1./5 for _ in range(5)], mode='same')
    gate3 = np.convolve(gate3, [1./5 for _ in range(5)], mode='same')

    gate0 = gate0 - gate0[nz:].mean(axis=0)
    gate1 = gate1 - gate1[nz:].mean(axis=0)
    gate2 = gate2 - gate2[nz:].mean(axis=0)
    gate3 = gate3 - gate3[nz:].mean(axis=0)
    
    gate0 = gate0.clip(0, 1)
    gate1 = gate1.clip(0, 1)
    gate2 = gate2.clip(0, 1)
    gate3 = gate3.clip(0, 1)

    gate0 = (gate0 - gate0[nz:].min(axis=0))/ (gate0[nz:].max(axis=0) - gate0[nz:].min(axis=0))
    gate1 = (gate1 - gate1[nz:].min(axis=0))/ (gate1[nz:].max(axis=0) - gate1[nz:].min(axis=0))
    gate2 = (gate2 - gate2[nz:].min(axis=0))/ (gate2[nz:].max(axis=0) - gate2[nz:].min(axis=0))
    gate3 = (gate3 - gate3[nz:].min(axis=0))/ (gate3[nz:].max(axis=0) - gate3[nz:].min(axis=0))



    X_words = [vec.idx2word[int(i)] for i in filters_pico['3'][s, :, 0]]

    for i, j in zip(X_words, gate0):
        if i != '[0]':
            print(color(i, bg=[int(255),int((1-j)*255), int((1-j)*255)]),)

    print("")
    for i, j in zip(X_words, gate1) :
        if i != '[0]':
            print(color(i, bg=[int((1-j)*255), int(255), int((1-j)*255)]),)

    print("")

    for i, j in zip(X_words, gate2) :
        if i != '[0]':
            print(color(i, bg=[int((1-j)*255), int((1-j)*255), int(255)]),)

    print("")
    for i, j in zip(X_words, gate3) :
        if i != '[0]':
            print(color(i, bg=[int(255), int(255), int((1-j)*255)]),)
            
    print("")
    #print "----------------------------------------------------------------------------------"+str(v)

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(int "")? (<ipython-input-18-4e125314b60b>, line 43)

In [30]:
activated_words = {}
for aspect in aspects :
    filters = filters_pico[aspect]
    word_activations = np.zeros((vec.vocab_size, 1))
    word2filter = filters.reshape(-1, filters.shape[-1])
    for word in word2filter :
        word_activations[int(word[0]), 0] = word[1]
    activated_words[aspect] = word_activations
    
joint = np.concatenate([activated_words[aspect] for aspect in aspects], axis=1)
joint = joint[2:, :]

In [31]:
top_words_df = pd.DataFrame()
top_words_size = 20
for aspect in aspects :
    popwords = joint[:, int(aspect)]
    sorted_idx = np.argsort(popwords)
    top_words = sorted_idx[-top_words_size:]
    print [str(v) for k, v in vec.idx2word.items() if k in top_words]
    top_words = pd.Series([v for k, v in vec.idx2word.items() if k in top_words])
    
    top_words_df[aspect] = top_words
top_words_df

['beans', 'bomb', 'christmas', 'complexity', 'disappointment', 'fingers', 'folks', 'good', 'holy', 'including', 'lemons', 'looked', 'metallic', 'mildly', 'nor', 'presentation', 'size', 'steady', 'time', 'waste']
['brownish', 'complete', 'drain', 'edge', 'everything', 'gone', 'interesting', 'into', 'lacing', 'malty', 'mexican', 'o', 'pine', 'play', 'qqq', 'representation', 'skunk', 'slowly', 'terms', 'want']
['amount', 'black', 'bomber', 'cut', 'd-', 'degrees', 'deserves', 'drink', 'english', 'excited', 'healthy', 'hops', 'mass', 'needs', 'nutty', 'previous', 'qqq', 'retains', 'thirst', 'worst']
['available', 'away', 'burnt', 'cheers', 'd-', 'down', 'english', 'enjoyable', 'entirely', 'except', 'falls', 'flavor', 'great', 'imo', 'medium-bodied', 'pleasantly', 'porters', 'qqq', 'residual', 'warmth']


,0,1,2,3
0,beans,brownish,amount,available
1,bomb,complete,black,away
2,christmas,drain,bomber,burnt
3,complexity,edge,cut,cheers
4,disappointment,everything,d-,d-
5,fingers,gone,degrees,down
6,folks,interesting,deserves,english
7,good,into,drink,enjoyable
8,holy,lacing,english,entirely
9,including,malty,excited,except


In [21]:
import spacy
nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])

In [22]:
s="Deep amber hue ,this brew is topped with a finger of off white head.\
Smell of dog hair, green olives, and slightly fruity.\
Taste of Belgian yeast, coriander, hard water and bready malt.\
Light body, with little carbonation."

In [23]:
t1 = [vec.word2idx[x.text.lower()] if x.text.lower() in vec.word2idx else 1 for x in nlp(unicode(s))]# if x in vec.word2idx]
t2 = [0]*(305-len(t1)) + t1
X = np.vstack((X[:-1], t2))
print [x.text.lower() if x.text.lower() in vec.word2idx else 1 for x in nlp(unicode(s))]

[u'deep', u'amber', u'hue', u',', u'this', u'brew', u'is', u'topped', u'with', u'a', u'finger', u'of', u'off', u'white', u'head', u'.', u'smell', u'of', u'dog', 1, u',', u'green', 1, u',', u'and', u'slightly', u'fruity', u'.', u'taste', u'of', u'belgian', u'yeast', u',', u'coriander', u',', u'hard', u'water', u'and', u'bready', u'malt', u'.', u'light', u'body', u',', u'with', u'little', u'carbonation', u'.']
